In [5]:
# importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] == 300

False

In [12]:
# import and merging the datasets
df1 = pd.read_csv("../Data/Part2 - Data1.csv")
df2 = pd.read_csv("../Data/Part2 -Data2.csv")
display(df1.shape,df2.shape)
df = pd.merge(df1, df2, on='ID', how='inner',
               validate="many_to_many")
display(df.shape)
# converting column names into lowercase
df.columns = df.columns.str.lower()

(5000, 8)

(5000, 7)

(5000, 14)

In [13]:
# glancing at the dataset
display(df.sample(5))

,id,age,customersince,highestspend,zipcode,hiddenscore,monthlyaveragespend,level,mortgage,security,fixeddepositaccount,internetbanking,creditcard,loanoncard
4060,4061,31,6,174,93023,2,6.7,1,0,0,0,1,1,0.0
1547,1548,47,21,52,94720,1,1.2,2,194,0,0,0,0,0.0
1654,1655,60,34,102,94305,2,2.0,1,0,0,0,1,1,0.0
1166,1167,30,5,112,91711,4,5.0,2,0,0,0,0,1,1.0
2411,2412,47,22,65,91330,3,2.7,2,0,0,0,1,0,0.0


In [14]:
#profile = ProfileReport(df, title='Dataframe Report')
#profile

Data Cleaning

In [16]:
# dropping the mising value
df.shape
df.dropna(inplace=True)
df.shape

(4980, 14)

In [17]:
# checking the distribution of target variable
def check_imbalance(df, class_column='class'):
    no_of_true = len(df.loc[df[class_column] == True])
    no_of_false = len(df.loc[df[class_column] == False])

    true_ratio = (no_of_true / (no_of_true + no_of_false))
    false_ratio = (no_of_false / (no_of_false + no_of_true))

    print(f"Number of true: {no_of_true} ({round(true_ratio, 4) * 100}%)")
    print(f"Number of false: {no_of_false} ({round(false_ratio, 5) * 100}%)")
check_imbalance(df,class_column='loanoncard')

Number of true: 480 (9.64%)
Number of false: 4500 (90.361%)


In [18]:
X = df.drop(['loanoncard','zipcode','id'], axis=1)
y = df['loanoncard']

# Splitting the dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)

In [19]:
random_forest = RandomForestClassifier(n_estimators=10, random_state=23)
random_forest.fit(X_train,y_train)

RandomForestClassifier(n_estimators=10, random_state=23)

In [20]:
# checking the accuracy score of a classifier on both train and test
print(f"Training set accuracy {random_forest.score(X_train, y_train)}")
print(f"Testing set accuracy {random_forest.score(X_test,y_test)}")

Training set accuracy 0.9991394148020654
Testing set accuracy 0.9825970548862115


In [21]:
n = [100,200,300]
d = [5,10,20,30,40]
l = [300,400,500]
seed = 1

def random_forest_tuning(x_train, x_test, y_train, y_test, n, d, l, seed):

    model_performance = []

    for i in n:
        for j in d:
            for k in l:
                # Create and train the Random Forest model
                random_forest = RandomForestClassifier(
                    n_estimators=i,
                    max_depth=j,
                    min_samples_leaf=k,
                    random_state=seed
                )
                random_forest.fit(x_train, y_train.ravel())

                # Predict probabilities on the training and testing sets
                train_pred = random_forest.predict_proba(x_train)
                test_pred = random_forest.predict_proba(x_test)

                # Create a unique identifier for the current set of hyperparameters
                t1 = f'trees{i}_maxDepth{j}_minLeaf{k}'

                # Calculate and store AUC-ROC scores for training and testing sets
                t2 = [t1, round(roc_auc_score(y_train, train_pred[:, 1]), 4), round(roc_auc_score(y_test, test_pred[:, 1]), 4)]
                model_performance.append(t2)

    # Create a DataFrame from the collected performance metrics
    model_performance_df = pd.DataFrame(model_performance)
    model_performance_df.rename(columns={0: 'parameter', 1: 'train_auc', 2: 'test_auc'}, inplace=True)

    return model_performance_df

model_performance_df = random_forest_tuning(n=n, d=d, l=l, seed=seed,
                     x_train=X_train,x_test=X_test,
                     y_train=y_train, y_test=y_test)

In [22]:
model_performance_df.head()

,parameter,train_auc,test_auc
0,trees100_maxDepth5_minLeaf300,0.9610,0.9434
1,trees100_maxDepth5_minLeaf400,0.9612,0.9444
2,trees100_maxDepth5_minLeaf500,0.9555,0.9460
3,trees100_maxDepth10_minLeaf300,0.9610,0.9434
4,trees100_maxDepth10_minLeaf400,0.9612,0.9444


2nd round of tuning

In [23]:
n = [50,100,150]
d = [5,6,7,8,9,10,20,30,40]
l = [100,200,225,250,275,300,325,350,375]
seed = 1

model_performance_df = random_forest_tuning(n=n, d=d, l=l, seed=seed,
                     x_train=X_train,x_test=X_test,
                     y_train=y_train, y_test=y_test)

In [24]:
model_performance_df.head()

,parameter,train_auc,test_auc
0,trees50_maxDepth5_minLeaf100,0.9869,0.9749
1,trees50_maxDepth5_minLeaf200,0.9748,0.9538
2,trees50_maxDepth5_minLeaf225,0.9732,0.9534
3,trees50_maxDepth5_minLeaf250,0.9662,0.9467
4,trees50_maxDepth5_minLeaf275,0.9634,0.9457


Best features

In [25]:
random_forest = RandomForestClassifier(n_estimators=50,
                                        max_depth=5,
                                        random_state=1,
                                        min_samples_leaf=100)
random_forest.fit(X_train, y_train)
train_pred = random_forest.predict_proba(X_train)
test_pred = random_forest.predict_proba(X_test)
print("train auc : ",round( roc_auc_score(y_train , train_pred[:,1]),4))
print("test auc : ",round( roc_auc_score(y_test , test_pred[:,1]),4))

train auc :  0.9869
test auc :  0.9749


In [26]:
def get_feature_importance(X_train, model, top_n=5):
    
    feature_importances = model.feature_importances_

    t1 = []
    for i in range(len(X_train.columns)):
        t2 = [X_train.columns[i], feature_importances[i]]
        t1.append(t2)

    imp_feat_df = pd.DataFrame(t1, columns=['name', 'importance'])
    imp_feat_df = imp_feat_df.sort_values(by=['importance'], ascending=False).head(top_n)

    return imp_feat_df
get_feature_importance(X_train,random_forest,5)

,name,importance
2,highestspend,0.478470
4,monthlyaveragespend,0.267936
5,level,0.122108
8,fixeddepositaccount,0.067449
6,mortgage,0.026159
